In [ ]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
import model
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import utils
import loop
import pickle
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
category = "balanced_data"
path = "../"
args = {
    "dims": [499, 256, 64, 32],
    "highly_genes": 500,
    'alpha': 0.001,
    'gamma': 0.001,
    'learning_rate': 0.0001,
    'update_epoch': 10,
    't_alpha': 1,
    'error': 0.001
}
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d
args = objectview(args)

files = glob2.glob(f'../R/simulated_data/{category}/*.h5')
files = [f[len(f"{path}R/simulated_data/{category}/"):-3] for f in files]
files

In [ ]:
dr = pd.DataFrame(
    columns=["dataset", "dropout", "perc0", "nb_genes", "exp", "ari"])
for run in range(1):
    df = pd.DataFrame()

    print(df.shape)
    for dataset in files:
        print(f">>>>> Data {dataset}")

        data_mat = h5py.File(f"{path}R/simulated_data/{category}/{dataset}.h5", "r")
        for nb_genes in [100, 200, 500, 1000, 1500]:
            dp = data_mat['dropout'][0]
            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            X = np.ceil(X).astype(np.int)
            count_X = X
            print(X.shape, count_X.shape)
            orig_X = X.copy()
            adata = sc.AnnData(X)
            adata.obs['Group'] = Y
            adata = utils.normalize(adata,
                                    copy=True,
                                    highly_genes=nb_genes,
                                    size_factors=True,
                                    normalize_input=True,
                                    logtrans_input=True)
            X = adata.X.astype(np.float32)
            Y = np.array(adata.obs["Group"])
            print(X.shape, count_X.shape)

            high_variable = np.array(adata.var.highly_variable.index,
                                     dtype=np.int)
            count_X = count_X[:, high_variable]
            # X=X[:, high_variable]
            size_factor = np.array(adata.obs.size_factors).reshape(
                -1, 1).astype(np.float32)
            cluster_number = int(max(Y) - min(Y) + 1)
            zeros = np.min(X, axis=0)
            X.shape, count_X.shape

            pxt = PCA(2).fit_transform(X)
            dresults = {
                "dataset": dataset,
                "nclust": cluster_number,
                "original": utils.evaluate(X, Y, cluster_number)[1],
                "pca": utils.evaluate(pxt, Y, cluster_number)[1],
                "perc_0": perc_0
            }

            # 3 models
            st_results = []

            f = []
            nb_zeros = int(0.8 * nb_genes)
            print(f"nb_zeros = {nb_zeros}, nb_genes= {nb_genes}")
            nm = nb_genes
            r1 = loop.self_train_clustering(X,
                                            Y,
                                            cluster_number,
                                            args,
                                            augm_zeros=None,
                                            nb_zeros=nb_zeros,
                                            random=False,
                                            perc=0.1,
                                            augm_value=0,
                                            model_name="STClustering",
                                            epochs=500)
            f.append(r1['features'])
            st_results.append(r1)

            dresults[f"self_initial_{nm}"] = r1["aris_kmeans_representation"][
                -1]
            dresults[f"self_{nm}"] = r1["aris"][-1]
            dr.loc[dr.shape[0]] = [
                dataset, dp, perc_0, nb_genes, "method1-kmeans",
                r1["aris_kmeans_representation"][-1]
            ]
            dr.loc[dr.shape[0]] = [
                dataset, dp, perc_0, nb_genes, "method1-leiden",
                r1["ari_leiden_representation"]
            ]
            
            
            dr.loc[dr.shape[0]] = [
                dataset, dp, perc_0, nb_genes, "method2", r1["aris"][-1]
            ]
            print("ARI ", r1["aris_kmeans_representation"][-1], " leiden ",
                 r1["ari_leiden_representation"])
            df = df.append(dresults, ignore_index=True)
            print(dresults)
            all_res = dresults.copy()
            all_res["st"] = st_results
            if os.path.isdir(f"{path}output/{category}/") == False:
                os.makedirs(f"{path}output/{category}/",exist_ok=True)
            with open(
                    f"{path}output/{category}/results_{dataset}_{run}_input_size.pickle",
                    'wb') as handle:
                pickle.dump(dresults, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(
                    f"{path}output/{category}/detailed_results_{dataset}_{run}_input_size.pickle",
                    'wb') as handle:
                pickle.dump(all_res, handle, protocol=pickle.HIGHEST_PROTOCOL)
            df.to_pickle(f"{path}output/pickle_results/{category}/real_data_{run}_input_size.pkl")
            dr.to_pickle(f"{path}output/pickle_results/{category}/real_data_input_size.pkl")

In [ ]:
dr = pd.read_pickle(f"{path}output/pickle_results/{category}/real_data_input_size.pkl")

In [ ]:
dr = dr.rename(columns = {"nb_genes": "Nb input genes"})

exp_names = {
    'method1-kmeans': "Baseline + KMeans", 
    'method1-leiden': "Baseline + Leiden", 
    'method2': "Cluster network"
}

dr["exp"] = dr["exp"].apply(lambda x: exp_names[x] if x in exp_names else x  )

# dr.to_pickle(f"{path}output/pickle_results/{category}/real_data_input_size.pkl")

In [ ]:
import seaborn as sns

plt.figure(figsize=(6, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="exp",
    data=dr,
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
sns.despine()
plt.legend(bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.xlabel("Method")
plt.savefig(f"{path}diagrams/balanced_input_size.pdf", bbox_inches='tight')